# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,14.44,56,0,3.67,RU,07/07/2024
1,1,waitangi,-43.9535,-176.5597,7.12,83,50,2.24,NZ,07/07/2024
2,2,uturoa,-16.7333,-151.4333,25.91,80,5,7.33,PF,07/07/2024
3,3,port-aux-francais,-49.3500,70.2167,2.10,83,100,12.03,TF,07/07/2024
4,4,pisco,-13.7000,-76.2167,20.03,73,0,3.60,PE,07/07/2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
).opts(title="City Humidity Heat Map")

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
city_low_temp = city_data_df.loc[city_data_df['Max Temp']>21,:]
city_high_temp = city_low_temp.loc[city_low_temp['Max Temp']<27,:]
city_wind_speed = city_high_temp.loc[city_high_temp['Wind Speed']<4.5,:]
city_cloudiness = city_wind_speed.loc[city_wind_speed['Cloudiness']==0,:]
city_low_humidity = city_cloudiness.loc[city_cloudiness['Humidity']>28,:]
city_high_humidity = city_low_humidity.loc[city_low_humidity['Humidity']<95,:]
city_high_humidity

# Drop any rows with null values
ideal_weather = city_high_humidity.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,66,constantia,44.1833,28.6500,22.61,66,0,2.29,RO,07/07/2024
212,212,arraial do cabo,-22.9661,-42.0278,24.92,78,0,2.57,BR,07/07/2024
232,232,velyka oleksandrivka,47.3197,33.3037,24.06,30,0,4.46,UA,07/07/2024
318,318,urumqi,43.8010,87.6005,21.53,64,0,2.00,CN,07/07/2024
329,329,hami,42.8000,93.4500,24.97,41,0,1.06,CN,07/07/2024
334,334,berkane,34.9084,-2.3461,26.82,45,0,2.86,MA,07/07/2024
336,336,prince george,53.9166,-122.7530,26.69,45,0,2.06,CA,07/07/2024
497,497,campos,-21.7500,-41.3000,26.03,65,0,4.12,BR,07/07/2024
519,519,kyzylorda,44.8528,65.5092,22.01,60,0,3.00,KZ,07/07/2024
522,522,eynesil,41.0645,39.1438,22.86,84,0,3.67,TR,07/07/2024


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
66,constantia,RO,44.1833,28.6500,66,
212,arraial do cabo,BR,-22.9661,-42.0278,78,
232,velyka oleksandrivka,UA,47.3197,33.3037,30,
318,urumqi,CN,43.8010,87.6005,64,
329,hami,CN,42.8000,93.4500,41,
334,berkane,MA,34.9084,-2.3461,45,
336,prince george,CA,53.9166,-122.7530,45,
497,campos,BR,-21.7500,-41.3000,65,
519,kyzylorda,KZ,44.8528,65.5092,60,
522,eynesil,TR,41.0645,39.1438,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey':geoapify_key,
    'categories':'accommodation.hotel'    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
constantia - nearest hotel: Hotel Dali
arraial do cabo - nearest hotel: No hotel found
velyka oleksandrivka - nearest hotel: No hotel found
urumqi - nearest hotel: 百时快捷酒店红山店
hami - nearest hotel: Хами
berkane - nearest hotel: Rosalina
prince george - nearest hotel: Travelodge by Wyndham Prince George
campos - nearest hotel: Hotel Terrazzo
kyzylorda - nearest hotel: Нур
eynesil - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
66,constantia,RO,44.1833,28.6500,66,Hotel Dali
212,arraial do cabo,BR,-22.9661,-42.0278,78,No hotel found
232,velyka oleksandrivka,UA,47.3197,33.3037,30,No hotel found
318,urumqi,CN,43.8010,87.6005,64,百时快捷酒店红山店
329,hami,CN,42.8000,93.4500,41,Хами
334,berkane,MA,34.9084,-2.3461,45,Rosalina
336,prince george,CA,53.9166,-122.7530,45,Travelodge by Wyndham Prince George
497,campos,BR,-21.7500,-41.3000,65,Hotel Terrazzo
519,kyzylorda,KZ,44.8528,65.5092,60,Нур
522,eynesil,TR,41.0645,39.1438,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_name_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
).opts(title="Hotel Location and Information Map")

# Display the map
hotel_name_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)